In [1]:
import time
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By

def start_stable_driver():
    options = Options()
    options.page_load_strategy = 'eager'
    options.add_argument("--disable-blink-features=AutomationControlled")
    prefs = {"profile.managed_default_content_settings.images": 2}
    options.add_experimental_option("prefs", prefs)
    driver = webdriver.Chrome(options=options)
    return driver

# Kategoriler ve Hedef Sayfalar
categories = {
    "dikey-supurge": 48,
    "derin-dondurucu": 49
}

all_category_links = {cat: [] for cat in categories}

for cat, last_page in categories.items():
    print(f"🚀 {cat} kategorisi için linkler toplanıyor...")
    driver = start_stable_driver()
    
    for p in range(1, last_page + 1):
        try:
            driver.get(f"https://www.sikayetvar.com/arcelik/{cat}?page={p}")
            time.sleep(2)
            links = driver.find_elements(By.CSS_SELECTOR, "a.complaint-layer")
            for l in links:
                href = l.get_attribute("href")
                title = l.get_attribute("title")
                if href and "/arcelik/" in href:
                    all_category_links[cat].append({"baslik": title, "url": href})
            
            if p % 10 == 0:
                print(f"✅ {cat}: Sayfa {p} bitti. Toplam: {len(all_category_links[cat])}")
        except:
            continue
    
    driver.quit()

print(f"🏁 Link toplama bitti! Süpürge: {len(all_category_links['dikey-supurge'])}, Dondurucu: {len(all_category_links['derin-dondurucu'])}")

🚀 dikey-supurge kategorisi için linkler toplanıyor...
✅ dikey-supurge: Sayfa 10 bitti. Toplam: 231
✅ dikey-supurge: Sayfa 20 bitti. Toplam: 366
✅ dikey-supurge: Sayfa 30 bitti. Toplam: 437
✅ dikey-supurge: Sayfa 40 bitti. Toplam: 494
🚀 derin-dondurucu kategorisi için linkler toplanıyor...
✅ derin-dondurucu: Sayfa 10 bitti. Toplam: 199
✅ derin-dondurucu: Sayfa 20 bitti. Toplam: 290
✅ derin-dondurucu: Sayfa 30 bitti. Toplam: 347
✅ derin-dondurucu: Sayfa 40 bitti. Toplam: 387
🏁 Link toplama bitti! Süpürge: 518, Dondurucu: 411


In [2]:
import requests
from bs4 import BeautifulSoup
import random
from concurrent.futures import ThreadPoolExecutor

def fetch_details_ultra(item):
    url = item['url']
    headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/121.0.0.0 Safari/537.36"}
    
    for attempt in range(4):
        try:
            time.sleep(random.uniform(0.5, 1.2))
            res = requests.get(url, headers=headers, timeout=15)
            if res.status_code == 200:
                soup = BeautifulSoup(res.text, 'html.parser')
                content = []
                
                # Yöntem 1: Modern
                start = soup.find("div", {"data-ga-element": "Complaint_Content_Start"})
                if start:
                    for sibling in start.find_next_siblings():
                        if sibling.name == "div" and sibling.get("data-ga-element") == "Complaint_Content_End": break
                        if sibling.name == "p": content.append(sibling.get_text(strip=True))
                
                # Yöntem 2: Klasik/Description
                if not content:
                    desc = soup.find("div", class_="complaint-detail-description")
                    if desc: content = [desc.get_text(strip=True)]
                
                full_text = " ".join(content)
                if len(full_text) > 10:
                    return {"baslik": item["baslik"], "url": url, "aciklama": full_text}
        except:
            continue
    return {"baslik": item["baslik"], "url": url, "aciklama": "HATA: Alınamadı."}

In [3]:
def process_category(cat_name, link_list):
    print(f"📥 {cat_name} detayları çekiliyor...")
    final_data = []
    with ThreadPoolExecutor(max_workers=6) as executor:
        futures = [executor.submit(fetch_details_ultra, link) for link in link_list]
        for i, future in enumerate(futures, 1):
            final_data.append(future.result())
            if i % 50 == 0:
                print(f"📊 {cat_name} İlerleme: {i}/{len(link_list)}")
    
    df = pd.DataFrame(final_data)
    df.to_excel(f"arcelik_{cat_name.replace('-', '_')}_final.xlsx", index=False)
    return df

# İşlemleri Başlat
df_supurge = process_category("dikey-supurge", all_category_links["dikey-supurge"])
df_dondurucu = process_category("derin-dondurucu", all_category_links["derin-dondurucu"])

print("✅ TÜM VERİLER KAYDEDİLDİ!")

📥 dikey-supurge detayları çekiliyor...
📊 dikey-supurge İlerleme: 50/518
📊 dikey-supurge İlerleme: 100/518
📊 dikey-supurge İlerleme: 150/518
📊 dikey-supurge İlerleme: 200/518
📊 dikey-supurge İlerleme: 250/518
📊 dikey-supurge İlerleme: 300/518
📊 dikey-supurge İlerleme: 350/518
📊 dikey-supurge İlerleme: 400/518
📊 dikey-supurge İlerleme: 450/518
📊 dikey-supurge İlerleme: 500/518
📥 derin-dondurucu detayları çekiliyor...
📊 derin-dondurucu İlerleme: 50/411
📊 derin-dondurucu İlerleme: 100/411
📊 derin-dondurucu İlerleme: 150/411
📊 derin-dondurucu İlerleme: 200/411
📊 derin-dondurucu İlerleme: 250/411
📊 derin-dondurucu İlerleme: 300/411
📊 derin-dondurucu İlerleme: 350/411
📊 derin-dondurucu İlerleme: 400/411
✅ TÜM VERİLER KAYDEDİLDİ!
